<a href="https://colab.research.google.com/github/bkrishr/ML-workbooks/blob/main/confusion_matrix_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Confusion Matrix and Model Evaluation
## Complete Guide with Real-World Case Studies

This notebook covers:
1. **Confusion Matrix Fundamentals**
2. **Metric Trade-offs** (Precision, Recall, F1-Score, Accuracy)
3. **Real-World Case Studies**
   - Medical Diagnosis (Cancer Detection)
   - Fraud Detection (Credit Card)
   - Email Spam Classification
4. **ROC & AUC Analysis**
5. **Balanced vs Imbalanced Dataset Strategies**
6. **Milestone-Based Improvement Plan**

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, classification_report,
    accuracy_score, precision_score, recall_score, f1_score,
    roc_curve, roc_auc_score, auc,
    precision_recall_curve, average_precision_score
)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import warnings
warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")

---
## 1. Understanding Confusion Matrix

### Confusion Matrix Components:

```
                 Predicted Positive    Predicted Negative
Actual Positive      TP                     FN
Actual Negative      FP                     TN
```

- **TP (True Positive)**: Correctly predicted positive
- **TN (True Negative)**: Correctly predicted negative
- **FP (False Positive)**: Type I Error - Predicted positive but actually negative
- **FN (False Negative)**: Type II Error - Predicted negative but actually positive

In [ ]:
def plot_confusion_matrix_detailed(y_true, y_pred, title="Confusion Matrix"):
    """
    Plot a detailed confusion matrix with metrics
    """
    cm = confusion_matrix(y_true, y_pred)

    # Calculate metrics
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Plot 1: Confusion Matrix Heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Predicted Negative', 'Predicted Positive'],
                yticklabels=['Actual Negative', 'Actual Positive'],
                cbar_kws={'label': 'Count'},
                annot_kws={'size': 20, 'weight': 'bold'},
                ax=axes[0])
    axes[0].set_title(title, fontsize=16, fontweight='bold', pad=20)
    axes[0].set_ylabel('Actual Class', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Predicted Class', fontsize=14, fontweight='bold')

    # Add labels to cells
    axes[0].text(0.5, 0.25, f'TN = {tn}', ha='center', va='center',
                fontsize=14, color='darkgreen', weight='bold')
    axes[0].text(1.5, 0.25, f'FP = {fp}\n(Type I Error)', ha='center', va='center',
                fontsize=14, color='darkred', weight='bold')
    axes[0].text(0.5, 1.25, f'FN = {fn}\n(Type II Error)', ha='center', va='center',
                fontsize=14, color='darkred', weight='bold')
    axes[0].text(1.5, 1.25, f'TP = {tp}', ha='center', va='center',
                fontsize=14, color='darkgreen', weight='bold')

    # Plot 2: Metrics Bar Chart
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
    values = [accuracy, precision, recall, f1]
    colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']

    bars = axes[1].bar(metrics, values, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    axes[1].set_ylim([0, 1.0])
    axes[1].set_ylabel('Score', fontsize=14, fontweight='bold')
    axes[1].set_title('Performance Metrics', fontsize=16, fontweight='bold', pad=20)
    axes[1].grid(axis='y', alpha=0.3)

    # Add value labels on bars
    for bar, value in zip(bars, values):
        height = bar.get_height()
        axes[1].text(bar.get_x() + bar.get_width()/2., height + 0.02,
                    f'{value:.3f}',
                    ha='center', va='bottom', fontsize=14, fontweight='bold')

    plt.tight_layout()
    plt.show()

    # Print detailed metrics
    print(f"\n{'='*60}")
    print(f"DETAILED METRICS FOR: {title}")
    print(f"{'='*60}")
    print(f"\nConfusion Matrix Values:")
    print(f"  True Positives (TP):  {tp:,}")
    print(f"  True Negatives (TN):  {tn:,}")
    print(f"  False Positives (FP): {fp:,} (Type I Error)")
    print(f"  False Negatives (FN): {fn:,} (Type II Error)")
    print(f"\nPerformance Metrics:")
    print(f"  Accuracy:  {accuracy:.4f} = (TP + TN) / Total")
    print(f"  Precision: {precision:.4f} = TP / (TP + FP)")
    print(f"  Recall:    {recall:.4f} = TP / (TP + FN)")
    print(f"  F1-Score:  {f1:.4f} = 2 * (Precision * Recall) / (Precision + Recall)")
    print(f"{'='*60}\n")

    return {'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp,
            'accuracy': accuracy, 'precision': precision,
            'recall': recall, 'f1': f1}

print("✅ Confusion matrix visualization function defined!")

---
## 2. Case Study 1: Medical Diagnosis (Cancer Detection)

### Scenario:
- **Problem**: Detecting cancer from medical scans
- **Dataset**: Highly imbalanced (2% cancer prevalence)
- **Critical**: Missing a cancer case (False Negative) can be fatal
- **Goal**: Maximize Recall (catch all cancer cases)

### Expected Behavior:
- **High Precision Model**: Few false alarms, but misses many cancers ❌
- **High Recall Model**: More false alarms, but catches most cancers ✅

In [ ]:
# Generate synthetic medical diagnosis data
np.random.seed(42)

# Simulate 10,000 patients with 2% cancer rate
n_samples = 10000
cancer_rate = 0.02

# Generate features (e.g., imaging features, biomarkers)
n_features = 20
X_medical = np.random.randn(n_samples, n_features)

# Generate labels (0 = healthy, 1 = cancer)
y_medical = np.random.choice([0, 1], size=n_samples, p=[1-cancer_rate, cancer_rate])

# Add signal to features for cancer cases
cancer_indices = np.where(y_medical == 1)[0]
X_medical[cancer_indices] += np.random.randn(len(cancer_indices), n_features) * 0.8

# Split data
X_train_med, X_test_med, y_train_med, y_test_med = train_test_split(
    X_medical, y_medical, test_size=0.3, random_state=42, stratify=y_medical
)

print("🏥 MEDICAL DIAGNOSIS DATASET")
print("="*60)
print(f"Total patients: {n_samples:,}")
print(f"Cancer cases: {np.sum(y_medical):,} ({100*cancer_rate:.1f}%)")
print(f"Healthy cases: {np.sum(y_medical==0):,} ({100*(1-cancer_rate):.1f}%)")
print(f"\nTrain set: {len(X_train_med):,} samples")
print(f"Test set: {len(X_test_med):,} samples")
print(f"Class distribution in test: Cancer={np.sum(y_test_med):,}, Healthy={np.sum(y_test_med==0):,}")
print("="*60)

In [ ]:
# Train models with different threshold strategies

# Model 1: High Precision (Conservative - fewer false alarms)
model_med_precision = RandomForestClassifier(n_estimators=100, random_state=42,
                                             class_weight={0:1, 1:5})  # Moderate weight
model_med_precision.fit(X_train_med, y_train_med)
y_pred_proba_prec = model_med_precision.predict_proba(X_test_med)[:, 1]
y_pred_precision = (y_pred_proba_prec > 0.7).astype(int)  # High threshold = High precision

# Model 2: High Recall (Aggressive - catch all cancers)
model_med_recall = RandomForestClassifier(n_estimators=100, random_state=42,
                                         class_weight={0:1, 1:20})  # Heavy weight on minority
model_med_recall.fit(X_train_med, y_train_med)
y_pred_proba_rec = model_med_recall.predict_proba(X_test_med)[:, 1]
y_pred_recall = (y_pred_proba_rec > 0.2).astype(int)  # Low threshold = High recall

print("✅ Medical diagnosis models trained!")
print("\nModel 1: High Precision Strategy (Threshold = 0.7)")
print("Model 2: High Recall Strategy (Threshold = 0.2)")

In [ ]:
# Evaluate High Precision Model
print("\n" + "#"*70)
print("# SCENARIO 1: HIGH PRECISION MODEL (Conservative)")
print("#"*70)
metrics_prec = plot_confusion_matrix_detailed(
    y_test_med, y_pred_precision,
    "Medical Diagnosis: High Precision Model"
)

print("\n⚠️  ANALYSIS:")
print(f"   - Only {metrics_prec['fp']} false alarms (good for patient stress)")
print(f"   - BUT MISSED {metrics_prec['fn']} CANCER CASES (DANGEROUS!)")
print(f"   - Recall of {metrics_prec['recall']:.1%} means we miss {100*(1-metrics_prec['recall']):.1f}% of cancers")
print(f"   - This is UNACCEPTABLE in medical diagnosis!")

In [ ]:
# Evaluate High Recall Model
print("\n" + "#"*70)
print("# SCENARIO 2: HIGH RECALL MODEL (Aggressive)")
print("#"*70)
metrics_rec = plot_confusion_matrix_detailed(
    y_test_med, y_pred_recall,
    "Medical Diagnosis: High Recall Model"
)

print("\n✅ ANALYSIS:")
print(f"   - {metrics_rec['fp']} false alarms (more stress, but manageable with follow-up tests)")
print(f"   - Only missed {metrics_rec['fn']} cancer cases (much better!)")
print(f"   - Recall of {metrics_rec['recall']:.1%} means we catch {100*metrics_rec['recall']:.1f}% of cancers")
print(f"   - This is PREFERRED in medical diagnosis - better safe than sorry!")

In [ ]:
# Compare the models side by side
comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'False Negatives', 'False Positives'],
    'High Precision\n(Conservative)': [
        f"{metrics_prec['accuracy']:.3f}",
        f"{metrics_prec['precision']:.3f}",
        f"{metrics_prec['recall']:.3f} ⚠️",
        f"{metrics_prec['f1']:.3f}",
        f"{metrics_prec['fn']} 🚨",
        f"{metrics_prec['fp']}"
    ],
    'High Recall\n(Aggressive)': [
        f"{metrics_rec['accuracy']:.3f}",
        f"{metrics_rec['precision']:.3f}",
        f"{metrics_rec['recall']:.3f} ✅",
        f"{metrics_rec['f1']:.3f}",
        f"{metrics_rec['fn']} ✅",
        f"{metrics_rec['fp']}"
    ]
})

print("\n" + "="*80)
print("MEDICAL DIAGNOSIS: MODEL COMPARISON")
print("="*80)
print(comparison_df.to_string(index=False))
print("="*80)
print("\n🎯 KEY TAKEAWAY FOR MEDICAL DIAGNOSIS:")
print("   → Prioritize RECALL over Precision")
print("   → Missing a cancer case (FN) is far worse than a false alarm (FP)")
print("   → False alarms can be resolved with follow-up tests")
print("   → Missed cancer can be fatal\n")

---
## 3. Case Study 2: Fraud Detection (Credit Card)

### Scenario:
- **Problem**: Detecting fraudulent credit card transactions
- **Dataset**: Extremely imbalanced (0.1% fraud rate)
- **Trade-off**:
  - High Recall: Catch fraud but block legitimate transactions (bad UX)
  - High Precision: Few false blocks but miss some fraud (financial loss)
- **Goal**: Balance between catching fraud and customer experience (F1-Score)

In [ ]:
# Generate synthetic fraud detection data
np.random.seed(42)

# Simulate 100,000 transactions with 0.1% fraud rate
n_transactions = 100000
fraud_rate = 0.001

# Generate features (transaction amount, time, location features, etc.)
n_features_fraud = 25
X_fraud = np.random.randn(n_transactions, n_features_fraud)

# Generate labels (0 = legitimate, 1 = fraud)
y_fraud = np.random.choice([0, 1], size=n_transactions, p=[1-fraud_rate, fraud_rate])

# Add strong signal to fraudulent transactions
fraud_indices = np.where(y_fraud == 1)[0]
X_fraud[fraud_indices] += np.random.randn(len(fraud_indices), n_features_fraud) * 1.5

# Split data
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(
    X_fraud, y_fraud, test_size=0.3, random_state=42, stratify=y_fraud
)

print("💳 CREDIT CARD FRAUD DETECTION DATASET")
print("="*60)
print(f"Total transactions: {n_transactions:,}")
print(f"Fraudulent: {np.sum(y_fraud):,} ({100*fraud_rate:.2f}%)")
print(f"Legitimate: {np.sum(y_fraud==0):,} ({100*(1-fraud_rate):.2f}%)")
print(f"\nTrain set: {len(X_train_fraud):,} transactions")
print(f"Test set: {len(X_test_fraud):,} transactions")
print(f"Fraud in test: {np.sum(y_test_fraud):,} cases")
print("="*60)

In [ ]:
# Use SMOTE to handle extreme imbalance
smote = SMOTE(random_state=42, sampling_strategy=0.1)
X_train_fraud_balanced, y_train_fraud_balanced = smote.fit_resample(X_train_fraud, y_train_fraud)

print(f"Original training data: {len(y_train_fraud):,} samples")
print(f"After SMOTE: {len(y_train_fraud_balanced):,} samples")
print(f"Fraud cases after SMOTE: {np.sum(y_train_fraud_balanced):,}")

# Model 1: High Precision (Conservative)
model_fraud_prec = GradientBoostingClassifier(n_estimators=100, random_state=42, max_depth=5)
model_fraud_prec.fit(X_train_fraud_balanced, y_train_fraud_balanced)
y_pred_proba_fraud_prec = model_fraud_prec.predict_proba(X_test_fraud)[:, 1]
y_pred_fraud_prec = (y_pred_proba_fraud_prec > 0.8).astype(int)  # High threshold

# Model 2: Balanced F1 (Optimal)
model_fraud_balanced = GradientBoostingClassifier(n_estimators=100, random_state=42, max_depth=5)
model_fraud_balanced.fit(X_train_fraud_balanced, y_train_fraud_balanced)
y_pred_proba_fraud_bal = model_fraud_balanced.predict_proba(X_test_fraud)[:, 1]
y_pred_fraud_bal = (y_pred_proba_fraud_bal > 0.3).astype(int)  # Optimized threshold

print("\n✅ Fraud detection models trained!")

In [ ]:
# Evaluate High Precision Model
print("\n" + "#"*70)
print("# SCENARIO 1: HIGH PRECISION MODEL (Conservative)")
print("#"*70)
metrics_fraud_prec = plot_confusion_matrix_detailed(
    y_test_fraud, y_pred_fraud_prec,
    "Fraud Detection: High Precision Model"
)

print("\n⚠️  ANALYSIS:")
print(f"   - Very few false blocks: {metrics_fraud_prec['fp']} legitimate transactions blocked")
print(f"   - BUT missed {metrics_fraud_prec['fn']} fraudulent transactions (financial loss!)")
print(f"   - Good customer experience, but significant fraud gets through")

In [ ]:
# Evaluate Balanced Model
print("\n" + "#"*70)
print("# SCENARIO 2: BALANCED F1 MODEL (Optimal)")
print("#"*70)
metrics_fraud_bal = plot_confusion_matrix_detailed(
    y_test_fraud, y_pred_fraud_bal,
    "Fraud Detection: Balanced F1 Model"
)

print("\n✅ ANALYSIS:")
print(f"   - {metrics_fraud_bal['fp']} false blocks (acceptable with good customer communication)")
print(f"   - Only {metrics_fraud_bal['fn']} frauds missed (much better protection!)")
print(f"   - Good balance: {100*metrics_fraud_bal['recall']:.1f}% fraud caught, "
      f"{100*metrics_fraud_bal['precision']:.1f}% accuracy when flagging")
print(f"   - F1-Score of {metrics_fraud_bal['f1']:.3f} indicates good balance")

In [ ]:
# Financial impact analysis
avg_fraud_amount = 150  # Average fraud transaction $150
avg_legit_amount = 75   # Average legitimate transaction $75

# High Precision Model costs
cost_missed_fraud_prec = metrics_fraud_prec['fn'] * avg_fraud_amount
cost_false_blocks_prec = metrics_fraud_prec['fp'] * 5  # $5 customer service cost per false block
total_cost_prec = cost_missed_fraud_prec + cost_false_blocks_prec

# Balanced Model costs
cost_missed_fraud_bal = metrics_fraud_bal['fn'] * avg_fraud_amount
cost_false_blocks_bal = metrics_fraud_bal['fp'] * 5
total_cost_bal = cost_missed_fraud_bal + cost_false_blocks_bal

print("\n" + "="*80)
print("FRAUD DETECTION: FINANCIAL IMPACT ANALYSIS")
print("="*80)
print("\nHigh Precision Model:")
print(f"  Cost of missed fraud: ${cost_missed_fraud_prec:,.2f}")
print(f"  Cost of false blocks: ${cost_false_blocks_prec:,.2f}")
print(f"  TOTAL COST: ${total_cost_prec:,.2f}")
print("\nBalanced F1 Model:")
print(f"  Cost of missed fraud: ${cost_missed_fraud_bal:,.2f}")
print(f"  Cost of false blocks: ${cost_false_blocks_bal:,.2f}")
print(f"  TOTAL COST: ${total_cost_bal:,.2f}")
print(f"\n💰 SAVINGS with Balanced Model: ${total_cost_prec - total_cost_bal:,.2f}")
print("="*80)
print("\n🎯 KEY TAKEAWAY FOR FRAUD DETECTION:")
print("   → Use F1-Score to balance fraud detection and customer experience")
print("   → Both Precision and Recall matter")
print("   → Optimize for total cost (fraud loss + customer friction)\n")

---
## 4. Case Study 3: Email Spam Classification

### Scenario:
- **Problem**: Classifying emails as spam or legitimate
- **Dataset**: Moderately balanced (40% spam)
- **Critical**: Blocking important emails (FP) is worse than letting spam through (FN)
- **Goal**: Maximize Precision (minimize false spam classifications)

In [ ]:
# Generate synthetic email spam data
np.random.seed(42)

# Simulate 20,000 emails with 40% spam rate
n_emails = 20000
spam_rate = 0.4

# Generate features (word counts, sender features, etc.)
n_features_email = 30
X_email = np.random.randn(n_emails, n_features_email)

# Generate labels (0 = legitimate, 1 = spam)
y_email = np.random.choice([0, 1], size=n_emails, p=[1-spam_rate, spam_rate])

# Add signal to spam emails
spam_indices = np.where(y_email == 1)[0]
X_email[spam_indices] += np.random.randn(len(spam_indices), n_features_email) * 1.0

# Split data
X_train_email, X_test_email, y_train_email, y_test_email = train_test_split(
    X_email, y_email, test_size=0.3, random_state=42, stratify=y_email
)

print("📧 EMAIL SPAM CLASSIFICATION DATASET")
print("="*60)
print(f"Total emails: {n_emails:,}")
print(f"Spam emails: {np.sum(y_email):,} ({100*spam_rate:.1f}%)")
print(f"Legitimate emails: {np.sum(y_email==0):,} ({100*(1-spam_rate):.1f}%)")
print(f"\nTrain set: {len(X_train_email):,} emails")
print(f"Test set: {len(X_test_email):,} emails")
print("\n⚖️ This is a MODERATELY BALANCED dataset")
print("="*60)

In [ ]:
# Model 1: High Recall (Aggressive spam filter)
model_email_recall = RandomForestClassifier(n_estimators=100, random_state=42)
model_email_recall.fit(X_train_email, y_train_email)
y_pred_proba_email_rec = model_email_recall.predict_proba(X_test_email)[:, 1]
y_pred_email_rec = (y_pred_proba_email_rec > 0.3).astype(int)  # Low threshold = High recall

# Model 2: High Precision (Conservative spam filter)
model_email_precision = RandomForestClassifier(n_estimators=100, random_state=42)
model_email_precision.fit(X_train_email, y_train_email)
y_pred_proba_email_prec = model_email_precision.predict_proba(X_test_email)[:, 1]
y_pred_email_prec = (y_pred_proba_email_prec > 0.7).astype(int)  # High threshold = High precision

print("✅ Email spam classification models trained!")

In [ ]:
# Evaluate High Recall Model
print("\n" + "#"*70)
print("# SCENARIO 1: HIGH RECALL MODEL (Aggressive Spam Filter)")
print("#"*70)
metrics_email_rec = plot_confusion_matrix_detailed(
    y_test_email, y_pred_email_rec,
    "Email Spam: High Recall Model"
)

print("\n❌ ANALYSIS:")
print(f"   - Catches most spam: only {metrics_email_rec['fn']} spam emails get through")
print(f"   - BUT {metrics_email_rec['fp']} IMPORTANT EMAILS BLOCKED (Bad user experience!)")
print(f"   - Users miss important messages from work, family, bills, etc.")
print(f"   - This creates frustration and distrust in the spam filter")

In [ ]:
# Evaluate High Precision Model
print("\n" + "#"*70)
print("# SCENARIO 2: HIGH PRECISION MODEL (Conservative Spam Filter)")
print("#"*70)
metrics_email_prec = plot_confusion_matrix_detailed(
    y_test_email, y_pred_email_prec,
    "Email Spam: High Precision Model"
)

print("\n✅ ANALYSIS:")
print(f"   - Only {metrics_email_prec['fp']} legitimate emails wrongly blocked (excellent!)")
print(f"   - {metrics_email_prec['fn']} spam emails get through (annoying but not critical)")
print(f"   - {100*metrics_email_prec['precision']:.1f}% of spam-flagged emails are actually spam")
print(f"   - Users can manually delete spam, but can't recover missed important emails")
print(f"   - This is PREFERRED for email - better to see some spam than miss important mail")

In [ ]:
# User experience impact
print("\n" + "="*80)
print("EMAIL SPAM: USER EXPERIENCE ANALYSIS")
print("="*80)
print("\nHigh Recall Model (Aggressive):")
print(f"  ✅ Spam caught: {metrics_email_rec['tp']} / {metrics_email_rec['tp'] + metrics_email_rec['fn']} "
      f"({100*metrics_email_rec['recall']:.1f}%)")
print(f"  ❌ Important emails blocked: {metrics_email_rec['fp']} (UNACCEPTABLE)")
print(f"  😠 User frustration: HIGH (missing important emails)")

print("\nHigh Precision Model (Conservative):")
print(f"  ✅ Important emails safe: {metrics_email_prec['tn']} / {metrics_email_prec['tn'] + metrics_email_prec['fp']} "
      f"({100*metrics_email_prec['tn']/(metrics_email_prec['tn'] + metrics_email_prec['fp']):.1f}%)")
print(f"  📧 Spam in inbox: {metrics_email_prec['fn']} (annoying but manageable)")
print(f"  😊 User satisfaction: HIGH (no important emails lost)")

print("="*80)
print("\n🎯 KEY TAKEAWAY FOR EMAIL SPAM:")
print("   → Prioritize PRECISION over Recall")
print("   → False Positive (blocking important email) is worse than False Negative (spam in inbox)")
print("   → Users can delete spam, but can't recover blocked important emails")
print("   → Trust and user experience are paramount\n")

---
## 5. ROC Curve and AUC Analysis

### ROC Curve (Receiver Operating Characteristic):
- Plots **True Positive Rate (TPR)** vs **False Positive Rate (FPR)**
- TPR = Recall = TP / (TP + FN)
- FPR = FP / (FP + TN)
- Shows model performance across all classification thresholds

### AUC (Area Under Curve):
- Single metric summarizing ROC curve
- Range: 0 to 1
- Interpretation:
  - 0.90-1.00: Excellent
  - 0.80-0.90: Good
  - 0.70-0.80: Fair
  - 0.60-0.70: Poor
  - 0.50: Random (no skill)

In [ ]:
def plot_roc_comparison(models_dict, X_test, y_test, title="ROC Curve Comparison"):
    """
    Plot ROC curves for multiple models

    models_dict: {'Model Name': (model, color)}
    """
    plt.figure(figsize=(14, 10))

    # Create subplots
    gs = plt.GridSpec(2, 2, hspace=0.3, wspace=0.3)
    ax1 = plt.subplot(gs[0, :])
    ax2 = plt.subplot(gs[1, 0])
    ax3 = plt.subplot(gs[1, 1])

    # Plot 1: ROC Curves
    for name, (model, color) in models_dict.items():
        # Get prediction probabilities
        if hasattr(model, 'predict_proba'):
            y_pred_proba = model.predict_proba(X_test)[:, 1]
        else:
            y_pred_proba = model.decision_function(X_test)

        # Calculate ROC curve
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
        roc_auc = auc(fpr, tpr)

        # Plot ROC curve
        ax1.plot(fpr, tpr, color=color, lw=3,
                label=f'{name} (AUC = {roc_auc:.3f})', alpha=0.8)

    # Plot diagonal (random classifier)
    ax1.plot([0, 1], [0, 1], 'k--', lw=2, label='Random Classifier (AUC = 0.500)', alpha=0.5)

    ax1.set_xlim([-0.02, 1.02])
    ax1.set_ylim([-0.02, 1.02])
    ax1.set_xlabel('False Positive Rate (FPR)', fontsize=14, fontweight='bold')
    ax1.set_ylabel('True Positive Rate (TPR) / Recall', fontsize=14, fontweight='bold')
    ax1.set_title(title, fontsize=16, fontweight='bold', pad=20)
    ax1.legend(loc='lower right', fontsize=12, framealpha=0.9)
    ax1.grid(alpha=0.3)

    # Add annotations
    ax1.annotate('Perfect Classifier\n(TPR=1, FPR=0)', xy=(0, 1), xytext=(0.3, 0.85),
                fontsize=11, ha='center',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgreen', alpha=0.7),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.3', lw=2))

    # Plot 2: AUC Bar Chart
    auc_scores = []
    model_names = []
    colors_list = []

    for name, (model, color) in models_dict.items():
        if hasattr(model, 'predict_proba'):
            y_pred_proba = model.predict_proba(X_test)[:, 1]
        else:
            y_pred_proba = model.decision_function(X_test)

        roc_auc = roc_auc_score(y_test, y_pred_proba)
        auc_scores.append(roc_auc)
        model_names.append(name)
        colors_list.append(color)

    bars = ax2.barh(model_names, auc_scores, color=colors_list, alpha=0.7, edgecolor='black', linewidth=2)
    ax2.set_xlim([0.5, 1.0])
    ax2.set_xlabel('AUC Score', fontsize=12, fontweight='bold')
    ax2.set_title('AUC Comparison', fontsize=14, fontweight='bold')
    ax2.grid(axis='x', alpha=0.3)

    # Add value labels
    for bar, score in zip(bars, auc_scores):
        width = bar.get_width()
        ax2.text(width + 0.01, bar.get_y() + bar.get_height()/2,
                f'{score:.4f}',
                ha='left', va='center', fontsize=11, fontweight='bold')

    # Add AUC interpretation zones
    ax2.axvspan(0.90, 1.0, alpha=0.2, color='green', label='Excellent')
    ax2.axvspan(0.80, 0.90, alpha=0.2, color='lightgreen', label='Good')
    ax2.axvspan(0.70, 0.80, alpha=0.2, color='yellow', label='Fair')
    ax2.axvspan(0.50, 0.70, alpha=0.2, color='orange', label='Poor')

    # Plot 3: Precision-Recall Curve
    for name, (model, color) in models_dict.items():
        if hasattr(model, 'predict_proba'):
            y_pred_proba = model.predict_proba(X_test)[:, 1]
        else:
            y_pred_proba = model.decision_function(X_test)

        precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
        avg_precision = average_precision_score(y_test, y_pred_proba)

        ax3.plot(recall, precision, color=color, lw=3,
                label=f'{name} (AP = {avg_precision:.3f})', alpha=0.8)

    ax3.set_xlim([-0.02, 1.02])
    ax3.set_ylim([-0.02, 1.02])
    ax3.set_xlabel('Recall', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Precision', fontsize=12, fontweight='bold')
    ax3.set_title('Precision-Recall Curve', fontsize=14, fontweight='bold')
    ax3.legend(loc='best', fontsize=10)
    ax3.grid(alpha=0.3)

    plt.tight_layout()
    plt.show()

print("✅ ROC and PR curve plotting function defined!")

In [ ]:
# Plot ROC curves for Medical Diagnosis models
print("\n" + "="*80)
print("ROC & AUC ANALYSIS: MEDICAL DIAGNOSIS")
print("="*80)

medical_models = {
    'High Precision Model': (model_med_precision, '#e74c3c'),
    'High Recall Model': (model_med_recall, '#2ecc71')
}

plot_roc_comparison(medical_models, X_test_med, y_test_med,
                   "ROC Curve: Medical Diagnosis Models")

In [ ]:
# Plot ROC curves for Fraud Detection models
print("\n" + "="*80)
print("ROC & AUC ANALYSIS: FRAUD DETECTION")
print("="*80)

fraud_models = {
    'High Precision Model': (model_fraud_prec, '#3498db'),
    'Balanced F1 Model': (model_fraud_balanced, '#f39c12')
}

plot_roc_comparison(fraud_models, X_test_fraud, y_test_fraud,
                   "ROC Curve: Fraud Detection Models")

In [ ]:
# Plot ROC curves for Email Spam models
print("\n" + "="*80)
print("ROC & AUC ANALYSIS: EMAIL SPAM CLASSIFICATION")
print("="*80)

email_models = {
    'High Recall Model': (model_email_recall, '#9b59b6'),
    'High Precision Model': (model_email_precision, '#1abc9c')
}

plot_roc_comparison(email_models, X_test_email, y_test_email,
                   "ROC Curve: Email Spam Classification Models")

---
## 6. Threshold Optimization

Demonstration of how changing classification threshold affects metrics

In [ ]:
def plot_threshold_analysis(model, X_test, y_test, title="Threshold Analysis"):
    """
    Plot how metrics change with different classification thresholds
    """
    # Get prediction probabilities
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Test different thresholds
    thresholds = np.linspace(0.1, 0.9, 50)
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)

        if len(np.unique(y_pred)) > 1:  # At least one of each class predicted
            accuracies.append(accuracy_score(y_test, y_pred))
            precisions.append(precision_score(y_test, y_pred, zero_division=0))
            recalls.append(recall_score(y_test, y_pred, zero_division=0))
            f1_scores.append(f1_score(y_test, y_pred, zero_division=0))
        else:
            accuracies.append(np.nan)
            precisions.append(np.nan)
            recalls.append(np.nan)
            f1_scores.append(np.nan)

    # Plot
    plt.figure(figsize=(16, 6))

    plt.plot(thresholds, accuracies, 'o-', label='Accuracy', linewidth=2, markersize=4, color='#3498db')
    plt.plot(thresholds, precisions, 's-', label='Precision', linewidth=2, markersize=4, color='#e74c3c')
    plt.plot(thresholds, recalls, '^-', label='Recall', linewidth=2, markersize=4, color='#2ecc71')
    plt.plot(thresholds, f1_scores, 'd-', label='F1-Score', linewidth=2, markersize=4, color='#f39c12')

    plt.xlabel('Classification Threshold', fontsize=14, fontweight='bold')
    plt.ylabel('Score', fontsize=14, fontweight='bold')
    plt.title(title, fontsize=16, fontweight='bold', pad=20)
    plt.legend(fontsize=12, loc='best')
    plt.grid(alpha=0.3)
    plt.ylim([0, 1.05])

    # Annotate key points
    # Find optimal F1 threshold
    if not all(np.isnan(f1_scores)):
        optimal_idx = np.nanargmax(f1_scores)
        optimal_threshold = thresholds[optimal_idx]
        optimal_f1 = f1_scores[optimal_idx]

        plt.axvline(x=optimal_threshold, color='red', linestyle='--', linewidth=2, alpha=0.5)
        plt.annotate(f'Optimal F1\nThreshold: {optimal_threshold:.2f}\nF1: {optimal_f1:.3f}',
                    xy=(optimal_threshold, optimal_f1), xytext=(optimal_threshold + 0.15, optimal_f1 - 0.15),
                    fontsize=11, ha='left',
                    bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7),
                    arrowprops=dict(arrowstyle='->', lw=2, color='red'))

    plt.tight_layout()
    plt.show()

    # Print insights
    print(f"\n{'='*80}")
    print(f"THRESHOLD ANALYSIS INSIGHTS")
    print(f"{'='*80}")
    print(f"\n📊 As threshold increases (0.1 → 0.9):")
    print(f"   • Precision typically INCREASES (fewer false positives)")
    print(f"   • Recall typically DECREASES (more false negatives)")
    print(f"   • Accuracy depends on class balance")
    print(f"   • F1-Score shows the optimal balance point")
    print(f"\n🎯 Choose threshold based on business requirements:")
    print(f"   • Low threshold (0.2-0.4): High Recall, lower Precision")
    print(f"   • Medium threshold (0.4-0.6): Balanced F1")
    print(f"   • High threshold (0.6-0.8): High Precision, lower Recall")
    print(f"{'='*80}\n")

print("✅ Threshold analysis function defined!")

In [ ]:
# Analyze threshold impact for Fraud Detection
plot_threshold_analysis(model_fraud_balanced, X_test_fraud, y_test_fraud,
                       "Threshold Analysis: Fraud Detection")

---
## 7. Balanced vs Imbalanced Dataset Strategy

### Summary of Metric Selection

| Dataset Type | Primary Metrics | Why |
|-------------|-----------------|-----|
| **Balanced** | Accuracy, F1-Score, ROC-AUC | All classes well represented |
| **Imbalanced** | PR-AUC, F1-Score, Recall/Precision | Accuracy is misleading |

### When to Use Each Metric:

1. **Accuracy**: Balanced datasets, overall correctness matters
2. **Precision**: When False Positives are costly (spam filter, expensive interventions)
3. **Recall**: When False Negatives are costly (disease detection, fraud)
4. **F1-Score**: When you need balance between Precision and Recall
5. **ROC-AUC**: Comparing models, balanced datasets
6. **PR-AUC**: Imbalanced datasets (better than ROC-AUC)

In [ ]:
# Create comparison visualization
def create_metric_selection_guide():
    """
    Visual guide for metric selection
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Metric Selection Decision Guide', fontsize=18, fontweight='bold', y=0.995)

    # Plot 1: Dataset Balance Decision
    ax1 = axes[0, 0]
    balance_categories = ['Balanced\n(40-60% each)', 'Slightly Imbalanced\n(20-40% minority)',
                         'Imbalanced\n(5-20% minority)', 'Highly Imbalanced\n(<5% minority)']
    recommended_metrics = ['Accuracy\nF1-Score', 'F1-Score\nRecall',
                          'Recall\nPR-AUC', 'Recall\nPR-AUC']
    colors_balance = ['#2ecc71', '#f39c12', '#e74c3c', '#c0392b']

    y_pos = np.arange(len(balance_categories))
    bars = ax1.barh(y_pos, [1]*len(balance_categories), color=colors_balance, alpha=0.7, edgecolor='black', linewidth=2)
    ax1.set_yticks(y_pos)
    ax1.set_yticklabels(balance_categories, fontsize=11)
    ax1.set_xlabel('Dataset Balance Level', fontsize=12, fontweight='bold')
    ax1.set_title('Step 1: Assess Dataset Balance', fontsize=14, fontweight='bold')
    ax1.set_xlim([0, 1.5])
    ax1.set_xticks([])

    for i, (bar, metric) in enumerate(zip(bars, recommended_metrics)):
        ax1.text(0.5, i, metric, ha='center', va='center', fontsize=11, fontweight='bold')

    # Plot 2: Cost of Errors Decision
    ax2 = axes[0, 1]
    error_types = ['FP > FN\n(Spam Filter)', 'FN > FP\n(Disease Detection)',
                  'FP ≈ FN\n(General Classification)', 'Both Critical\n(Safety Systems)']
    metric_choice = ['Precision', 'Recall', 'F1-Score', 'Maximize\nBoth']
    colors_error = ['#3498db', '#e74c3c', '#f39c12', '#9b59b6']

    bars2 = ax2.barh(np.arange(len(error_types)), [1]*len(error_types),
                     color=colors_error, alpha=0.7, edgecolor='black', linewidth=2)
    ax2.set_yticks(np.arange(len(error_types)))
    ax2.set_yticklabels(error_types, fontsize=11)
    ax2.set_xlabel('Error Cost Scenario', fontsize=12, fontweight='bold')
    ax2.set_title('Step 2: Evaluate Error Costs', fontsize=14, fontweight='bold')
    ax2.set_xlim([0, 1.5])
    ax2.set_xticks([])

    for i, (bar, metric) in enumerate(zip(bars2, metric_choice)):
        ax2.text(0.5, i, metric, ha='center', va='center', fontsize=12, fontweight='bold', color='white')

    # Plot 3: Use Case Examples
    ax3 = axes[1, 0]
    use_cases = [
        ('Medical\nDiagnosis', 'Recall', '#e74c3c'),
        ('Fraud\nDetection', 'F1-Score', '#f39c12'),
        ('Spam\nFilter', 'Precision', '#3498db'),
        ('Credit\nScoring', 'PR-AUC', '#9b59b6'),
        ('Quality\nControl', 'Recall', '#e74c3c')
    ]

    use_case_names = [uc[0] for uc in use_cases]
    use_case_metrics = [uc[1] for uc in use_cases]
    use_case_colors = [uc[2] for uc in use_cases]

    bars3 = ax3.bar(range(len(use_cases)), [1]*len(use_cases),
                    color=use_case_colors, alpha=0.7, edgecolor='black', linewidth=2)
    ax3.set_xticks(range(len(use_cases)))
    ax3.set_xticklabels(use_case_names, fontsize=11, rotation=0)
    ax3.set_ylabel('Priority Metric', fontsize=12, fontweight='bold')
    ax3.set_title('Step 3: Real-World Use Cases', fontsize=14, fontweight='bold')
    ax3.set_ylim([0, 1.3])
    ax3.set_yticks([])

    for i, (bar, metric) in enumerate(zip(bars3, use_case_metrics)):
        ax3.text(i, 0.5, metric, ha='center', va='center', fontsize=11, fontweight='bold', color='white')

    # Plot 4: Metric Trade-offs
    ax4 = axes[1, 1]

    tradeoff_text = [
        "Precision ↑ → Recall ↓",
        "Recall ↑ → Precision ↓",
        "F1 balances both",
        "Accuracy misleads on\nimbalanced data",
        "ROC-AUC: balanced data",
        "PR-AUC: imbalanced data"
    ]

    ax4.axis('off')
    ax4.set_title('Step 4: Understand Trade-offs', fontsize=14, fontweight='bold', pad=20)

    y_text = 0.9
    for i, text in enumerate(tradeoff_text):
        color = ['#e74c3c', '#2ecc71', '#f39c12', '#c0392b', '#3498db', '#9b59b6'][i]
        ax4.text(0.5, y_text, text, ha='center', va='center', fontsize=13,
                fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.8', facecolor=color, alpha=0.3, edgecolor=color, linewidth=2))
        y_text -= 0.15

    plt.tight_layout()
    plt.show()

create_metric_selection_guide()
print("\n✅ Metric selection guide created!")

---
## 8. Milestone-Based Model Improvement Strategy

Step-by-step approach to improving model performance

In [ ]:
# Demonstrate milestone-based improvement on fraud detection
print("\n" + "="*80)
print("MILESTONE-BASED IMPROVEMENT: FRAUD DETECTION EXAMPLE")
print("="*80)

# Baseline Model (Phase 1)
print("\n" + "#"*80)
print("# PHASE 1: BASELINE MODEL")
print("#"*80)
baseline_model = LogisticRegression(random_state=42, max_iter=1000)
baseline_model.fit(X_train_fraud, y_train_fraud)
y_pred_baseline = baseline_model.predict(X_test_fraud)

metrics_baseline = plot_confusion_matrix_detailed(
    y_test_fraud, y_pred_baseline,
    "Phase 1: Baseline Logistic Regression"
)

baseline_f1 = metrics_baseline['f1']
baseline_recall = metrics_baseline['recall']
print(f"\n📊 Baseline Metrics: F1={baseline_f1:.3f}, Recall={baseline_recall:.3f}")

In [ ]:
# Phase 2: Threshold Optimization
print("\n" + "#"*80)
print("# PHASE 2: THRESHOLD OPTIMIZATION")
print("#"*80)

y_pred_proba_baseline = baseline_model.predict_proba(X_test_fraud)[:, 1]

# Find optimal threshold for F1
best_f1 = 0
best_threshold = 0.5
for threshold in np.linspace(0.1, 0.9, 50):
    y_pred_thresh = (y_pred_proba_baseline >= threshold).astype(int)
    if len(np.unique(y_pred_thresh)) > 1:
        f1 = f1_score(y_test_fraud, y_pred_thresh)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

y_pred_phase2 = (y_pred_proba_baseline >= best_threshold).astype(int)
metrics_phase2 = plot_confusion_matrix_detailed(
    y_test_fraud, y_pred_phase2,
    f"Phase 2: Optimized Threshold ({best_threshold:.2f})"
)

improvement_phase2 = ((metrics_phase2['f1'] - baseline_f1) / baseline_f1) * 100
print(f"\n📈 Improvement: +{improvement_phase2:.1f}% in F1-Score")
print(f"   Strategy: Adjusted threshold from 0.5 to {best_threshold:.2f}")

In [ ]:
# Phase 3: Handle Class Imbalance with SMOTE
print("\n" + "#"*80)
print("# PHASE 3: CLASS IMBALANCE HANDLING (SMOTE)")
print("#"*80)

# Already balanced earlier, use those results
phase3_model = LogisticRegression(random_state=42, max_iter=1000)
phase3_model.fit(X_train_fraud_balanced, y_train_fraud_balanced)
y_pred_proba_phase3 = phase3_model.predict_proba(X_test_fraud)[:, 1]
y_pred_phase3 = (y_pred_proba_phase3 >= best_threshold).astype(int)

metrics_phase3 = plot_confusion_matrix_detailed(
    y_test_fraud, y_pred_phase3,
    "Phase 3: SMOTE + Optimized Threshold"
)

improvement_phase3 = ((metrics_phase3['f1'] - baseline_f1) / baseline_f1) * 100
print(f"\n📈 Cumulative Improvement: +{improvement_phase3:.1f}% in F1-Score")
print(f"   Strategy: SMOTE oversampling + threshold optimization")

In [ ]:
# Phase 4: Advanced Model (Gradient Boosting)
print("\n" + "#"*80)
print("# PHASE 4: ADVANCED MODEL (Gradient Boosting)")
print("#"*80)

# Use the already trained gradient boosting model
y_pred_proba_phase4 = model_fraud_balanced.predict_proba(X_test_fraud)[:, 1]
y_pred_phase4 = (y_pred_proba_phase4 >= best_threshold).astype(int)

metrics_phase4 = plot_confusion_matrix_detailed(
    y_test_fraud, y_pred_phase4,
    "Phase 4: Gradient Boosting + SMOTE + Optimized Threshold"
)

improvement_phase4 = ((metrics_phase4['f1'] - baseline_f1) / baseline_f1) * 100
print(f"\n📈 Total Improvement: +{improvement_phase4:.1f}% in F1-Score")
print(f"   Strategy: Advanced algorithm + SMOTE + threshold tuning")

In [ ]:
# Visualize milestone progression
phases = ['Phase 1\nBaseline', 'Phase 2\nThreshold', 'Phase 3\nSMOTE', 'Phase 4\nGBM']
f1_scores = [baseline_f1, metrics_phase2['f1'], metrics_phase3['f1'], metrics_phase4['f1']]
recalls = [baseline_recall, metrics_phase2['recall'], metrics_phase3['recall'], metrics_phase4['recall']]
precisions = [metrics_baseline['precision'], metrics_phase2['precision'],
             metrics_phase3['precision'], metrics_phase4['precision']]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Metric progression
x = np.arange(len(phases))
width = 0.25

bars1 = ax1.bar(x - width, f1_scores, width, label='F1-Score', color='#f39c12', alpha=0.8, edgecolor='black')
bars2 = ax1.bar(x, recalls, width, label='Recall', color='#2ecc71', alpha=0.8, edgecolor='black')
bars3 = ax1.bar(x + width, precisions, width, label='Precision', color='#e74c3c', alpha=0.8, edgecolor='black')

ax1.set_ylabel('Score', fontsize=14, fontweight='bold')
ax1.set_xlabel('Improvement Phase', fontsize=14, fontweight='bold')
ax1.set_title('Milestone-Based Performance Improvement', fontsize=16, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(phases, fontsize=11)
ax1.legend(fontsize=12)
ax1.set_ylim([0, 1.0])
ax1.grid(axis='y', alpha=0.3)

# Add value labels
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9, fontweight='bold')

# Plot 2: Cumulative improvement
improvements = [(f1 - baseline_f1) / baseline_f1 * 100 for f1 in f1_scores]
colors_improvement = ['#95a5a6', '#3498db', '#f39c12', '#2ecc71']

bars_imp = ax2.bar(phases, improvements, color=colors_improvement, alpha=0.8, edgecolor='black', linewidth=2)
ax2.set_ylabel('F1-Score Improvement (%)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Improvement Phase', fontsize=14, fontweight='bold')
ax2.set_title('Cumulative F1-Score Improvement', fontsize=16, fontweight='bold')
ax2.axhline(y=0, color='red', linestyle='--', linewidth=2, alpha=0.5)
ax2.grid(axis='y', alpha=0.3)

for bar, imp in zip(bars_imp, improvements):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 1,
            f'+{imp:.1f}%',
            ha='center', va='bottom', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("MILESTONE SUMMARY")
print("="*80)
print(f"Phase 1 (Baseline):          F1 = {baseline_f1:.3f}")
print(f"Phase 2 (+ Threshold):       F1 = {metrics_phase2['f1']:.3f} (+{improvements[1]:.1f}%)")
print(f"Phase 3 (+ SMOTE):           F1 = {metrics_phase3['f1']:.3f} (+{improvements[2]:.1f}%)")
print(f"Phase 4 (+ GBM):             F1 = {metrics_phase4['f1']:.3f} (+{improvements[3]:.1f}%)")
print("="*80)
print("\n🎯 KEY LESSONS:")
print("   1. Start simple (baseline) to establish metrics")
print("   2. Optimize threshold first (low-cost, high-impact)")
print("   3. Address class imbalance (SMOTE, class weights)")
print("   4. Try advanced algorithms (ensemble methods)")
print("   5. Iterate and measure improvement at each step\n")

---
## 9. Final Summary and Best Practices

### Key Takeaways:

1. **No Single Metric is Perfect**
   - Always consider multiple metrics
   - Context and business requirements matter

2. **Precision-Recall Trade-off**
   - Increasing Precision typically decreases Recall
   - Increasing Recall typically decreases Precision
   - F1-Score finds the balance

3. **Dataset Balance Matters**
   - Balanced: Accuracy is reliable
   - Imbalanced: Accuracy is misleading, use Recall/Precision/F1/PR-AUC

4. **Domain-Specific Choices**
   - Medical: Prioritize Recall (don't miss diseases)
   - Spam: Prioritize Precision (don't block important emails)
   - Fraud: Balance with F1-Score

5. **ROC vs PR Curves**
   - ROC-AUC: Good for balanced datasets
   - PR-AUC: Better for imbalanced datasets

6. **Improvement Strategy**
   - Start with baseline
   - Optimize threshold
   - Handle class imbalance
   - Try advanced algorithms
   - Measure at each milestone

In [ ]:
# Create final summary visualization
print("\n" + "="*80)
print("CONFUSION MATRIX & MODEL EVALUATION - COMPLETE GUIDE")
print("="*80)
print("\n✅ Notebook completed successfully!")
print("\nYou've learned:")
print("   • How to interpret confusion matrices")
print("   • Trade-offs between Precision, Recall, F1, and Accuracy")
print("   • Real-world case studies (Medical, Fraud, Spam)")
print("   • ROC and AUC analysis")
print("   • Metric selection for balanced vs imbalanced datasets")
print("   • Milestone-based improvement strategies")
print("\n🎯 Next Steps:")
print("   1. Apply these concepts to your own datasets")
print("   2. Experiment with different thresholds")
print("   3. Try various class balancing techniques")
print("   4. Always validate with domain experts")
print("\n" + "="*80)